<a href="https://colab.research.google.com/github/gauss5930/Natural-Language-Processing/blob/main/ELMo/character_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchtext
from torchtext.data import NestedField
import math

class BPTTIterator(torchtext.data.BPTTIterator):
  def __iter__(self):
    text = self.dataset[0].text
    TEXT = self.dataset.fields['text']
    TEXT.eos_token = None
    text = text + ([TEXT.pad_token] * int(math.ceil(len(text) / self.batch_size) * self.batch_size - len(text)))
    data = TEXT.pad([text])   #new
    data = TEXT.numericalize(data, device = self.device)

    #new line start
    size = list(data.size())
    size[0] = self.batch_size
    size[1] = -1

    data = data.view(*size).transpose(0, 1).contiguous()
    dataset = torchtext.data.Dataset(examples = self.dataset.examples, fields = [('text', 'TEXT'), ('target', 'TEXT')])

    while True:
      for i in range(0, len(self) * self.bptt_len, self.bptt_len):
        self.ierations += 1
        seq_len = min(self.bptt_len, len(data) - i - 1)
        batch_text = data[i:i + seq_len]
        if TEXT.batch_first:
          batch_text = batch_text.transpose(0, 1).contiguous()
          batch_target = batch_target.transpose(0, 1).contiguous()
        yield torchtext.data.Batch.fromvars(
            dataset, self.batch_size, text = batch_text, target = batch_target
        )
      if not self.repeat:
        return

def gen_bptt_iter(dataset, batch_size, bptt_len, device):
  #dataset: tuple of dataset
  for batch_word, batch_char in zip(
      BPTTIterator(dataset[0], batch_size, bptt_len, device = device),
      BPTTIterator(dataset[1], batch_size, bptt_len, device = device),
  ):
    yield batch_word.text, batch_char.text, batch_word.target, batch_char.target

def gen_language_model_corpus(dataset_cls: torchtext.datasets.LanguageModelingDataset):
  field_char = NestedField(Field(pad_token = PAD_WORD, tokenize = list, init_token = SOS_WORD,
                                 eos_token = EOS_WORD, batch_first = True), pad_token = PAD_WORD,)
  
  field_word = Field(batch_first = True)
  dataset_char = dataset_cls.splits(field_char)
  dataset_word = dataset_cls.splits(dielf_word)
  field_char.build_vocab(dataset_char[0])
  field_word.build_vocab(dataset_char[0])
  return [_ for _ in zip(dataset_word, dataset_char)], field_word, field_char

#How to use
if __name__ == '__main__':
  from torchtext.dataset import WIkiText2
  from torchtext.data import Field

  #FINAL
  PAD_WORD = '<pad>'
  SOS_WORD = '<sow>'
  EOS_WORD = '<eow>'

  datasets, field_word, field_char = gen_language_model_corpus(WikiText2)
  train_data, valid_data, test_data = datasets